# Domácí úkol:
Pomocí vlastních tříd a dědičnosti vytvořte "soubor tříd" = nástroj pro derivaci funkcí.

## Jak to má fungovat:

- derivace je jednoduše algoritmizovatelná, neboť se řídí prakticky pouze třemi pravidly (podílu se dá vyhnout pomocí násobení funkcí $\frac{1}{f(x)}$):
    - derivace součtu je součet derivací
    - pravidlo pro derivaci součinu
    - derivace funkce je derivovaná funkce násobená derivovaným argumentem

- tedy stačí implementovat tyto tři základní pravidla a pro každou elementární funkci (sin, cos, exp, ln, x^n, ...) vědět jak vypadá její derivace
- celý tento proces je možné reprezentovat pomocí tříd a dědičnosti
- trik je v tom, že se výrazy automaticky skládají do stromu (pro nás v podstatě skrytého), a tedy každý výraz je na venek pouze buď sčítání, násobení, nebo elementární funkce (a o jejich argumentech platí totéž)

## Podrobnější zadání:
Konkrétněji:
- Vytvořte třídu **Operator**
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor (zde stačí `pass`)
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
- Vytvořte třídu **BinaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `levy_argument` - levý operand
        - `pravy_argument` - pravý operand
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argumenty
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru
- Vytvořte třídu **UnaryOperator**, která bude dědit od třídy Operator
    - Třída bude mít atributy:
        - `argument` - argument funkce 
    - Třída bude mít metody:
        - `__init__()` - inicializuje operátor, přiřadí argument
        - `__str__()` - vrací řetězec reprezentující operátor
        - `__repr__()` - vrací řetězec reprezentující operátor
        - `diff()` - vrací derivaci operátoru

Pro počítání derivací použijte standardní pravidla:
- derivace součtu je součet derivací
- pravidlo pro derivaci součinu
- derivace funkce je derivovaná funkce násobená derivovaným argumentem

Pro **BinaryOperator** a **UnaryOperator** implementujte také metody implementující standardní operátory:
- `__add__()` - implementuje operátor `+`
- `__sub__()` - implementuje operátor `-`
- `__mul__()` - implementuje operátor `*`

Pro **BinaryOperator** implementujte dvě dceřiné třídy:
- **Addition** - implementuje operátor `+`
- **Multiplication** - implementuje operátor `*`

Pro **UnaryOperator** implementujte tyto dceřiné třídy:
- **exp** - implementuje funkci `exp`
- **sin** - implementuje funkci `sin`
- **cos** - implementuje funkci `cos`
- **ln** - implementuje funkci `ln`
- **frc** - implementuje funkci `1/x`
- **identity** - implementuje identitu `x`
- **cst** - implementuje konstantu `c`
- **mocnina** - implementuje mocninu `x^n`

U všech funckí implementujte adekvátně metodu `diff()` a `__str__()`, tak aby se funkce vypisovaly v přehledné podobě.

Ozkoušejte na následujících funkcích:
- $sin(x + cos(x*x)) + 1$
- $x^2 + 2x + 1 + 2x^2$
- $x^2 + 2x + exp(x)$

In [27]:
# kostra k doplnění, místa označená TODO doplňte

class Operator:
    def __init__(self):
        pass


    
    def __str__(self):
        return type(self).__name__
    


    def __repr__(self):
        return self.__str__()
    

    
    def diff(self):
        pass


    
    def __add__(self, other):
        return Addition(self, other)
    

        
    def __sub__(self, other):
        return Addition(self, Multiplication(cst(-1), other))
        
        
    
    def __mul__(self, other):
        return Multiplication(self, other)


        
    
class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        super().__init__()
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument


        
class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument

    
    def __str__(self):
        return type(self).__name__ + "(" + str(self.argument) + ")"
    


        
class Addition(BinaryOperator):
    def __str__(self):

        return f"({str(self.levy_argument)}) + ({str(self.pravy_argument)})"
        # return self.levy_argument + self.pravy_argument     


    def diff(self):
        return Addition(self.levy_argument.diff(), self.pravy_argument.diff())
    



    
class Multiplication(BinaryOperator):
    def __str__(self):
        return f"({str(self.levy_argument)}) * ({str(self.pravy_argument)})"
    
        # return self.levy_argument * self.pravy_argument  
    


    def diff(self):
        return Addition(
            Multiplication(self.levy_argument.diff(), self.pravy_argument),
            Multiplication(self.levy_argument, self.pravy_argument.diff())
        )


        
    
class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value


    def __str__(self):
        return str(self.value)
        

    def diff(self):
        return cst(0)
       
    
class identity(UnaryOperator):
    def __init__(self):
        pass


    def __str__(self):
        return "x"


    def diff(self):
        return cst(1)
    

    
class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent


    def __str__(self):
        return f"{self.argument} ^ {self.exponent}"

    def diff(self):
        return Multiplication(
            Multiplication(cst(self.exponent), mocnina(self.argument, self.exponent - 1)),
            self.argument.diff()
        )
    
    
class sin(UnaryOperator):
    def diff(self):
        return Multiplication(cos(self.argument), self.argument.diff())
    


class cos(UnaryOperator):
    def diff(self):
        return Multiplication(cst(-1), Multiplication(sin(self.argument), self.argument.diff()))
    


class exp(UnaryOperator):
    def diff(self):
        return Multiplication(exp(self.argument), self.argument.diff())
    

    
class ln(UnaryOperator):
    def diff(self):
        return Multiplication(frc(self.argument), self.argument.diff())


class frc(UnaryOperator):
    def __str__(self):
       return f"1/{self.argument}"
    
    def diff(self):
        return Multiplication(Multiplication(cst(-1), frc(mocnina(self.argument , 2))), self.argument.diff())

In [28]:
# takto by se to mělo používat
x = identity()
# sin(x + cos(x*x)) + 1
f1 = sin(x + cos(x*x)) + cst(1)
f1_derivative = f1.diff()



print(f"Function 1: {f1}")
print(f"Derivative 1: {f1_derivative}\n")


# print(f1)
# print(f1.diff())

Function 1: (sin((x) + (cos((x) * (x))))) + (1)
Derivative 1: ((cos((x) + (cos((x) * (x))))) * ((1) + ((-1) * ((sin((x) * (x))) * (((1) * (x)) + ((x) * (1))))))) + (0)



můj výstup byl:

`sin(x + cos((x)*(x))) + 1`

`(cos(x + cos((x)*(x))))*(1 + ((-1)*(sin((x)*(x))))*((1)*(x) + (x)*(1))) + 0`

In [29]:
x = identity()
# x ^ 2 + 2x + 1 + 2x ^ 2
f2 = mocnina(x,2) + cst(2)*x + cst(1) + cst(2)*mocnina(x,2)

f2_derivative = f2.diff()




print(f"Function 2: {f2}")
print(f"Derivative 2: {f2_derivative}\n")


# print(f2)
# print(f2.diff())


Function 2: (((x ^ 2) + ((2) * (x))) + (1)) + ((2) * (x ^ 2))
Derivative 2: (((((2) * (x ^ 1)) * (1)) + (((0) * (x)) + ((2) * (1)))) + (0)) + (((0) * (x ^ 2)) + ((2) * (((2) * (x ^ 1)) * (1))))



můj výstup byl:

`(x)^2 + (2)*(x) + 1 + (2)*((x)^2)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + 0 + (0)*((x)^2) + (2)*(((2)*((x)^1))*(1))`

In [21]:
x = identity()
# x ^ 2 + 2x + exp(x)
f3 = mocnina(x,2) + cst(2)*x + exp(x)
f3_derivative = f3.diff()


print(f"Function 3: {f3}")
print(f"Derivative 3: {f3_derivative}\n")



# print(f3)
# print(f3.diff())


Function 3: ((x ^ 2) + ((2) * (x))) + (exp(x))
Derivative 3: ((((2) * (x ^ 1)) * (1)) + (((0) * (x)) + ((2) * (1)))) + ((exp(x)) * (1))



můj výstup byl:

`(x)^2 + (2)*(x) + exp(x)`

`((2)*((x)^1))*(1) + (0)*(x) + (2)*(1) + (exp(x))*(1)`


In [30]:
# print(f"Function 1: {f1}")
# print(f"Derivative 1: {f1_derivative}\n")


# print(f"Function 2: {f2}")
# print(f"Derivative 2: {f2_derivative}\n")


# print(f"Function 3: {f3}")
# print(f"Derivative 3: {f3_derivative}\n")



# Bonusový úkol:
Doplňte ke všem třídám metodu `simplify()`, která provede zjednodušení výrazu. Například:
- součet s nulou vrátí druhý operand
- součin s nulou vrátí nulu
- mocnina s exponentem 0 vrátí 1
- mocnina s exponentem 1 vrátí první argument
- součín dvou stejných argumentů vrátí mocninu s exponentem 2
- součin konstant vrátí konstantu se součinem
- ...

Doplňte metody `__add__, __radd__, __mul__, __rmul__, ...` tak aby podporovaly operace s čísly (např. `2 + x` nebo `x * 3`), a zjednodušovaly zadávání výrazů. 

In [31]:


class Operator:
    def __add__(self, other):
        if isinstance(other, (int, float)):  
            other = cst(other)
        return Addition(self, other).simplify()

    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, other):
        if isinstance(other, (int, float)): 
            other = cst(other)
        return Multiplication(self, other).simplify()

    def __rmul__(self, other):
        return self.__mul__(other)

    def simplify(self):
        return self 

class BinaryOperator(Operator):
    def __init__(self, levy_argument, pravy_argument):
        self.levy_argument = levy_argument
        self.pravy_argument = pravy_argument

class UnaryOperator(Operator):
    def __init__(self, argument):
        self.argument = argument

    def __str__(self):
        return f"{type(self).__name__}({self.argument})"

class Addition(BinaryOperator):
    def __str__(self):
        return f"({self.levy_argument}) + ({self.pravy_argument})"
    
    def diff(self):
        return Addition(self.levy_argument.diff(), self.pravy_argument.diff()).simplify()
    
    def simplify(self):
        levy_simplified = self.levy_argument.simplify()
        pravy_simplified = self.pravy_argument.simplify()

        if isinstance(levy_simplified, cst) and levy_simplified.value == 0:
            return pravy_simplified
        if isinstance(pravy_simplified, cst) and pravy_simplified.value == 0:
            return levy_simplified

        return Addition(levy_simplified, pravy_simplified)

class Multiplication(BinaryOperator):
    def __str__(self):
        return f"({self.levy_argument}) * ({self.pravy_argument})"
    
    def diff(self):
        return Addition(
            Multiplication(self.levy_argument.diff(), self.pravy_argument),
            Multiplication(self.levy_argument, self.pravy_argument.diff())
        ).simplify()

    def simplify(self):
        levy_simplified = self.levy_argument.simplify()
        pravy_simplified = self.pravy_argument.simplify()

        if isinstance(levy_simplified, cst) and levy_simplified.value == 1:
            return pravy_simplified
        if isinstance(pravy_simplified, cst) and pravy_simplified.value == 1:
            return levy_simplified

        if isinstance(levy_simplified, cst) and levy_simplified.value == 0:
            return cst(0)
        if isinstance(pravy_simplified, cst) and pravy_simplified.value == 0:
            return cst(0)

        return Multiplication(levy_simplified, pravy_simplified)

class cst(UnaryOperator):
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return str(self.value)
    
    def diff(self):
        return cst(0)
    
    def simplify(self):
        return self  

class identity(UnaryOperator):
    def __init__(self):
        pass
    
    def __str__(self):
        return "x"
    
    def diff(self):
        return cst(1)

class mocnina(UnaryOperator):
    def __init__(self, argument, exponent):
        self.argument = argument
        self.exponent = exponent
    
    def __str__(self):
        return f"({self.argument})^{self.exponent}"
    
    def diff(self):
        return Multiplication(
            Multiplication(cst(self.exponent), mocnina(self.argument, self.exponent - 1)),
            self.argument.diff()
        ).simplify()

    def simplify(self):
        argument_simplified = self.argument.simplify()

        if self.exponent == 0:
            return cst(1)
        if self.exponent == 1:
            return argument_simplified

        return mocnina(argument_simplified, self.exponent)

class sin(UnaryOperator):
    def diff(self):
        return Multiplication(cos(self.argument), self.argument.diff()).simplify()

class cos(UnaryOperator):
    def diff(self):
        return Multiplication(cst(-1), Multiplication(sin(self.argument), self.argument.diff())).simplify()

class exp(UnaryOperator):
    def diff(self):
        return Multiplication(exp(self.argument), self.argument.diff()).simplify()

class ln(UnaryOperator):
    def diff(self):
        return Multiplication(frc(self.argument), self.argument.diff()).simplify()

class frc(UnaryOperator):
    def __str__(self):
        return f"1/({self.argument})"
    
    def diff(self):
        return Multiplication(Multiplication(cst(-1), frc(mocnina(self.argument, 2))), self.argument.diff()).simplify()



x = identity()

# sin(x + cos(x*x)) + 1
f1_simplified = sin(x + cos(x*x)) + 1
f1_derivative_simplified = f1_simplified.diff().simplify()

# x^2 + 2x + 1 + 2x^2
f2_simplified = mocnina(x, 2) + 2*x + 1 + 2*mocnina(x, 2)
f2_derivative_simplified = f2_simplified.diff().simplify()

# x^2 + 2x + exp(x)
f3_simplified = mocnina(x, 2) + 2*x + exp(x)
f3_derivative_simplified = f3_simplified.diff().simplify()


print(f"Function 1 (simplified): {f1_simplified}")
print(f"Derivative 1 (simplified): {f1_derivative_simplified}\n")

print(f"Function 2 (simplified): {f2_simplified}")
print(f"Derivative 2 (simplified): {f2_derivative_simplified}\n")

print(f"Function 3 (simplified): {f3_simplified}")
print(f"Derivative 3 (simplified): {f3_derivative_simplified}")


Function 1 (simplified): (sin((x) + (cos((x) * (x))))) + (1)
Derivative 1 (simplified): (cos((x) + (cos((x) * (x))))) * ((1) + ((-1) * ((sin((x) * (x))) * ((x) + (x)))))

Function 2 (simplified): ((((x)^2) + ((x) * (2))) + (1)) + (((x)^2) * (2))
Derivative 2 (simplified): (((2) * (x)) + (2)) + (((2) * (x)) * (2))

Function 3 (simplified): (((x)^2) + ((x) * (2))) + (exp(x))
Derivative 3 (simplified): (((2) * (x)) + (2)) + (exp(x))
